In [ ]:
import os

os.getcwd()

In [ ]:
%cd voxceleb_trainer

In [ ]:
%pip install -r requirements.txt

In [ ]:
!python3 ./dataprep.py --save_path ??? --download --user ??? --password ???  

In [ ]:
!python3 ./dataprep.py --save_path /home/jupyter/work/resources/voxceleb_trainer/data --extract
!python3 ./dataprep.py --save_path /home/jupyter/work/resources/voxceleb_trainer/data --convert

In [ ]:
#!g1.1
!python3 ./trainSpeakerNet.py --eval --model ResNetSE34V2 --log_input True --encoder_type ASP --n_mels 64 --trainfunc softmaxproto --save_path /home/jupyter/work/resources/voxceleb_trainer/exps/test --eval_frames 400  --initial_model baseline_v2_ap.model


## Генерируем семьи

In [ ]:
import torch

embs = torch.load('embeddings.pt')

In [ ]:
#делаем словарь вида {'id' : усредненный embedding диктора}

utterance_list = open('./data/train_list.txt', 'r')

mean_embs = dict()
current_id = ''
emb_sum, count = 0, 0

for line in utterance_list:
    id, path = line.split()
    
    if id == current_id:
        emb_sum += embs[path]
        count += 1
    else:
        if current_id:
            mean_embs[current_id] = 1 / count * emb_sum

        emb_sum = embs[path]
        count = 1
        current_id = id
        
    mean_embs[current_id] = 1 / count * emb_sum
        
utterance_list.close()

In [ ]:
#!g1.1
#score каждого диктора с каждым
import numpy
import torch.nn.functional as F

scores = dict()
ids = list(mean_embs.keys())

for i in range(len(ids)):
    for j in range(i + 1, len(ids)):
        
        ref_feat = mean_embs[ids[i]].cuda()
        com_feat = mean_embs[ids[j]].cuda()

        dist = F.cosine_similarity(ref_feat.unsqueeze(-1), com_feat.unsqueeze(-1).transpose(0,2)).detach().cpu().numpy();
        score = numpy.mean(dist);
        scores[(ids[i], ids[j])] = score


In [ ]:
similarity_scores = list(scores.values())
min_score = min(similarity_scores)
max_score = max(similarity_scores)
print(min_score, max_score)

In [ ]:
#!g1.1
#делаем матрицу scores

import numpy as np
import torch.nn.functional as F

ids = list(mean_embs.keys())

n = len(ids)
scores = np.zeros((n, n))
id2ind = {ids[i] : i for i in range(n)}

for i in range(n):
    for j in range(i + 1, n):
        
        ref_feat = mean_embs[ids[i]].cuda()
        com_feat = mean_embs[ids[j]].cuda()

        dist = F.cosine_similarity(ref_feat.unsqueeze(-1), com_feat.unsqueeze(-1).transpose(0,2)).detach().cpu().numpy();
        score = np.mean(dist);
        scores[id2ind[ids[i]], id2ind[ids[j]]] = score


In [ ]:
scores = scores + scores.T

In [ ]:
threshold = 0.3
ind2id = {i : ids[i] for i in range(n)}

components_mat = scores > threshold

In [ ]:
def generate_family(member_id, member_count, components_mat, id2ind, ind2id, relatives_set=set(ids)):
    
    family = {member_id}
    
    while len(family) < len(relatives_set):
        related = np.nonzero(components_mat[id2ind[member_id], :])[0]
        related_ids = [ind2id[i] for i in related]

        relatives_set &= set(related_ids)
        
        if len(family) == member_count:
            return family
        if not relatives_set:
            return family
        
        add_member_id = np.random.choice(list(relatives_set))
        
        #print('current member ', member_id, ' related to ', relatives_set, ' chosen connection to ', add_member_id)
        
        family.add(add_member_id)
        member_id = add_member_id
        
    return family


In [ ]:
families = []

for member_count in range(2, 8):
    fixed_size_families = []
    for id_ in ids:
        family = generate_family(id_, member_count, components_mat.astype(int), id2ind, ind2id, relatives_set=set(ids))
        if len(family) == member_count:
            fixed_size_families.append(family)
    families.append(fixed_size_families)

In [ ]:
#generate_family('id10104', components_mat.astype(int), id2ind, ind2id, relatives_set=set(ids))

In [ ]:
for i, fixed_families in enumerate(families):
    print('nSpeakers', i + 2, len(np.unique(fixed_families)))

## Метрики

In [ ]:
def tuneThresholdfromScore(scores, labels, target_fa, target_fr = None):
    
    fpr, tpr, thresholds = metrics.roc_curve(labels, scores, pos_label=1)
    fnr = 1 - tpr

    tunedThreshold = [];
    if target_fr:
        for tfr in target_fr:
            idx = numpy.nanargmin(numpy.absolute((tfr - fnr)))
            tunedThreshold.append([thresholds[idx], fpr[idx], fnr[idx]]);
    
    for tfa in target_fa:
        idx = numpy.nanargmin(numpy.absolute((tfa - fpr))) # numpy.where(fpr<=tfa)[0][-1]
        tunedThreshold.append([thresholds[idx], fpr[idx], fnr[idx]]);
    
    idxE = numpy.nanargmin(numpy.absolute((fnr - fpr)))
    eer  = max(fpr[idxE],fnr[idxE])*100
    
    return (tunedThreshold, eer, fpr, fnr);

# Creates a list of false-negative rates, a list of false-positive rates
# and a list of decision thresholds that give those error-rates.


def ComputeErrorRates(scores, labels):

      # Sort the scores from smallest to largest, and also get the corresponding
      # indexes of the sorted scores.  We will treat the sorted scores as the
      # thresholds at which the the error-rates are evaluated.
      sorted_indexes, thresholds = zip(*sorted(
          [(index, threshold) for index, threshold in enumerate(scores)],
          key=itemgetter(1)))
      sorted_labels = []
      labels = [labels[i] for i in sorted_indexes]
      fnrs = []
      fprs = []

      # At the end of this loop, fnrs[i] is the number of errors made by
      # incorrectly rejecting scores less than thresholds[i]. And, fprs[i]
      # is the total number of times that we have correctly accepted scores
      # greater than thresholds[i].
      for i in range(0, len(labels)):
          if i == 0:
              fnrs.append(labels[i])
              fprs.append(1 - labels[i])
          else:
              fnrs.append(fnrs[i-1] + labels[i])
              fprs.append(fprs[i-1] + 1 - labels[i])
      fnrs_norm = sum(labels)
      fprs_norm = len(labels) - fnrs_norm

      # Now divide by the total number of false negative errors to
      # obtain the false positive rates across all thresholds
      fnrs = [x / float(fnrs_norm) for x in fnrs]

      # Divide by the total number of corret positives to get the
      # true positive rate.  Subtract these quantities from 1 to
      # get the false positive rates.
      fprs = [1 - x / float(fprs_norm) for x in fprs]
      return fnrs, fprs, thresholds


In [ ]:
def false_acceptance_rate(labels, labels_true):
    labels, labels_true = np.array(labels), np.array(labels_true)
    return ((labels != 'g') & (labels_true == 'g')).sum() / len(labels) 


def false_rejection_rate(labels, labels_true):
    labels, labels_true = np.array(labels), np.array(labels_true)
    return ((labels == 'g') & (labels_true != 'g')).sum() / len(labels) 


def wrong_member_rate(labels, labels_true):
    labels, labels_true = np.array(labels), np.array(labels_true)
    return ((labels != 'g') & (labels_true != 'g') & (labels != labels_true)).sum() / (labels_true != 'g').sum() 


In [ ]:
#сделаем словарь вида {id : [list of path to records]}

f = open('./data/train_list.txt', 'r')
id2records = {ids[i] : [] for i in range(len(ids))}

for line in f:
    id_, path = line.split()
    id2records[id_].append(path)
f.close()

In [ ]:
#сделаем пробную генерацию выборки и пропустим ее через tuneThresholdfromScore
#необходимо сделать так, чтобы эмбеддинги из enrolls не пересекались с эмбеддингами из dataset

#member_id - person id 
#count - len(list of utterances from person_id)

def sample_utterance(member_id, count, except_paths=[]):
    path_records = list(np.random.choice(list(set(id2records[member_id]) - set(except_paths)), count))
    sample = [embs[path] for path in path_records]
    return sample, path_records

def generate_enrolls(family_ids, level=4):
    enrolls = dict.fromkeys(family_ids)
    for member_id in family_ids:
        utterances, path_records = sample_utterance(member_id, level)
        enrolls[member_id] = sum(utterances) / level
    return enrolls, path_records
    
from sklearn.utils import shuffle  
def generate_dataset(family_ids, enrolled_paths, ut_per_member=10, ut_per_guest=4, guests_count=4):
    utterances, labels, used_records = [], [], []
    for member_id in family_ids:
        sample, paths = sample_utterance(member_id, ut_per_member, enrolled_paths)
        
        utterances += sample
        used_records += paths
        labels += [member_id] * ut_per_member
        
    guest_ids = np.random.choice(list(set(ids) - set(family_ids)), guests_count)
    for guest_id in guest_ids:
        sample, paths = sample_utterance(guest_id, ut_per_guest, enrolled_paths)
        utterances += sample
        used_records += paths
        labels += ['g'] * ut_per_guest
    
    utterances, labels = shuffle(utterances, labels)
    return utterances, labels, used_records
    

In [ ]:
#!g1.1
#берем по 100 примеров каждой семьи
import torch.nn.functional as F

mean_eers = []

for i in range(len(families)):
    eers = []
    for j in range(1000):
        fam = list(families[i][j])
        enrolls, enrolls_path = generate_enrolls(fam)
        utts, labels_true, _ = generate_dataset(fam, enrolls_path)

        scores = []
        labels = []

        for utterance in utts:
            scores_with_enrolled = []
            for enrolled_id, enrolled_ut in enrolls.items():
                ref_feat = utterance.cuda()
                com_feat = enrolled_ut.cuda()
                dist = F.cosine_similarity(ref_feat.unsqueeze(-1), com_feat.unsqueeze(-1).transpose(0,2)).detach().cpu().numpy();
                score = numpy.mean(dist);
                scores_with_enrolled.append(score)

            scores.append(max(scores_with_enrolled))
            labels.append(list(enrolls.keys())[np.argmax(scores_with_enrolled)])


        far, frr, wmr = [], [], []

        thresholds = sorted(scores)
        labels, scores = np.array(labels), np.array(scores)

        for t in thresholds:
            labels[np.where(scores <= t)[0]] = 'g'
            far.append(false_acceptance_rate(labels, labels_true))
            frr.append(false_rejection_rate(labels, labels_true))
            wmr.append(wrong_member_rate(labels, labels_true))

        fnir, far = np.array(frr) + np.array(wmr), np.array(far)
        idxE = np.nanargmin(np.absolute((far - fnir)))
        eer  = max(far[idxE],fnir[idxE])*100
        eers.append(eer)
    mean_eers.append(np.mean(eers))

In [ ]:
#по 100 семей
for i in range(len(mean_eers)):
    print('nSpeakers =', i + 2, 'EER =', mean_eers[i])

In [ ]:
#по 1000 семей
for i in range(len(mean_eers)):
    print('nSpeakers =', i + 2, 'EER =', mean_eers[i])

In [ ]:
import torch.nn.functional as F

def cosine_score(emb_1, emb_2):
    ref_feat = emb_1.cuda()
    com_feat = emb_2.cuda()
    dist = F.cosine_similarity(ref_feat.unsqueeze(-1), com_feat.unsqueeze(-1).transpose(0,2)).detach().cpu().numpy();
    score = numpy.mean(dist);
    return score

In [ ]:
#!g1.1

simulations = 100
grid_search = []

for a in np.linspace(0.75, 0.95, 3):
    for threshold in np.linspace(0.4, 0.8, 5):

        mean_eers = []
        for i in range(len(families)):
            eers = []
            for j in range(simulations):
                fam = list(families[i][j])

                enrolls, enrolls_path = generate_enrolls(fam)
                train_utts, _, used_in_train = generate_dataset(fam, enrolls_path)
                utts, labels_true, _ = generate_dataset(fam, enrolls_path + used_in_train)

                #training enrolls
                for utterance in train_utts:
                    for enrolled_id, enrolled_ut in enrolls.items():

                        if cosine_score(utterance, enrolled_ut) > threshold:
                            enrolls[enrolled_id] = F.normalize(a * enrolled_ut + (1 - a) * utterance, p=2, dim=1)

                #metrics on test dataset

                scores, labels = [], []

                for utterance in utts:
                    scores_with_enrolled = []

                    for enrolled_id, enrolled_ut in enrolls.items():
                        scores_with_enrolled.append(cosine_score(utterance, enrolled_ut))

                    scores.append(max(scores_with_enrolled))
                    labels.append(list(enrolls.keys())[np.argmax(scores_with_enrolled)])


                far, frr, wmr = [], [], []

                thresholds = sorted(scores)
                labels, scores = np.array(labels), np.array(scores)

                for t in thresholds:
                    labels[np.where(scores <= t)[0]] = 'g'
                    far.append(false_acceptance_rate(labels, labels_true))
                    frr.append(false_rejection_rate(labels, labels_true))
                    wmr.append(wrong_member_rate(labels, labels_true))

                fnir, far = np.array(frr) + np.array(wmr), np.array(far)
                idxE = np.nanargmin(np.absolute((far - fnir)))
                eer  = max(far[idxE],fnir[idxE])*100

                eers.append(eer)

            mean_eers.append(np.mean(eers))

        grid_search.append(mean_eers)
        
        print(a, threshold, grid_search[-1], sep='  |  ')


In [ ]:
i = 0
for a in np.linspace(0.75, 0.95, 3):
    for threshold in np.linspace(0.4, 0.8, 5):
        print('a =', a, 'threshold =', threshold)
        print(grid_search[i])
        print()
        i += 1

In [ ]:
#!g1.1

a, threshold = 0.85, 0.7
simulations = 1000


mean_eers = []
for i in range(len(families)):
    eers = []
    for j in range(simulations):
        fam = list(families[i][j])

        enrolls, enrolls_path = generate_enrolls(fam)
        train_utts, _, used_in_train = generate_dataset(fam, enrolls_path)
        utts, labels_true, _ = generate_dataset(fam, enrolls_path + used_in_train)

        #training enrolls
        for utterance in train_utts:
            for enrolled_id, enrolled_ut in enrolls.items():

                if cosine_score(utterance, enrolled_ut) > threshold:
                    enrolls[enrolled_id] = F.normalize(a * enrolled_ut + (1 - a) * utterance, p=2, dim=1)

        #metrics on test dataset

        scores, labels = [], []

        for utterance in utts:
            scores_with_enrolled = []

            for enrolled_id, enrolled_ut in enrolls.items():
                scores_with_enrolled.append(cosine_score(utterance, enrolled_ut))

            scores.append(max(scores_with_enrolled))
            labels.append(list(enrolls.keys())[np.argmax(scores_with_enrolled)])


        far, frr, wmr = [], [], []

        thresholds = sorted(scores)
        labels, scores = np.array(labels), np.array(scores)

        for t in thresholds:
            labels[np.where(scores <= t)[0]] = 'g'
            far.append(false_acceptance_rate(labels, labels_true))
            frr.append(false_rejection_rate(labels, labels_true))
            wmr.append(wrong_member_rate(labels, labels_true))

        fnir, far = np.array(frr) + np.array(wmr), np.array(far)
        idxE = np.nanargmin(np.absolute((far - fnir)))
        eer  = max(far[idxE],fnir[idxE])*100

        eers.append(eer)

    mean_eers.append(np.mean(eers))


In [ ]:
#по 1000 семей
for i in range(len(mean_eers)):
    print('nSpeakers =', i + 2, 'EER =', mean_eers[i])

In [ ]:
train = open('./data/train_list.txt', 'r')
count_utterances = 0

for i in train:
    count_utterances += 1
train.close()

In [ ]:
import numpy as np

train = open('./data/train_list.txt', 'r')
test = open('./data/test_list.txt', 'a')

i = 0
buffer = ''

for line in train:
    if i % 2:
        id_1, path_1 = buffer.split()
        id_2, path_2 = line.split()
        test.write(str(int(id_1 == id_2)) + ' ' + path_1 + ' ' + path_2 + '\n')
    else:
        buffer = line
    i += 1
    
train.close()
test.close()

In [ ]:
#generates train_list [id, path] to each utterance 

path = './data/voxceleb1'
f = open('./data/train_list.txt', 'a')

for id in os.listdir(path):
    for youtube in os.listdir(path + '/' + id):
        for video_id in os.listdir(path + '/' + id + '/' + youtube):
            f.write(id + ' ' + id + '/' + youtube + '/' + video_id + '\n')
f.close()